**Вариант 1**
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
   * Наименование вакансии.
   * Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
   * Ссылку на саму вакансию.
   * Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
# !pip install bs4

In [2]:
# импорт библиотек
import requests
from pprint import pprint
from bs4 import BeautifulSoup as bs
import pandas as pd

In [3]:
url_hh = 'https://hh.ru/search/vacancy'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 YaBrowser/23.1.2.931 Yowser/2.5 Safari/537.36'}
resp = requests.get(url_hh, headers=headers)

In [36]:

# Функции преобразования данных
def salary_prep(salary):
    '''Преобразование данных по заработной плате'''
    try:
        salary = salary.text
    except:
        salary = None
    if salary == None:
        salary_min = None
        salary_max = None
        salary_currency = None
    else: 
        salary = salary.replace("\u202f", "") 
        salary = salary.replace("\xa0", "")
        salary_currency = None
        salary_list = salary.split(' ')
        if salary_list[0] == 'от':
            salary_min = int(salary_list[1])
            salary_max = None
            salary_currency = salary_list[-1]
        elif salary_list[0] == 'до':
            salary_min = None
            salary_max = int(salary_list[1])
            salary_currency = salary_list[-1]
        else:
            salary_min = int(salary_list[0])
            salary_max = int(salary_list[2])
            salary_currency = salary_list[-1]
#     salary_min = float(salary_min)
#     salary_max = float(salary_max)
   

    
    salary_list = [salary_min, salary_max, salary_currency]    
        
    return salary_list

# Функция замены закодированных пробелов в тексте
def text_prep(text):
    text = text.replace("\xa0", " ")
    text_list = text.split(',')
    if text_list[0] == None:
        text_list[0] = "None"
    return text_list[0]

In [37]:
# Функция парсинга HH
def get_conten_hh(resp): 
    soup = bs(resp.text, 'html.parser')
    items = soup.find_all('div', {'class':'vacancy-serp-item__layout'})
    hh_vacancy = []

    for item in items:
        salary = item.find('span', {'class':'bloko-header-section-3'})
        hh_vacancy.append(
            {
                'site': 'HeadHanter', # Название сайта
                'title': item.find('a', {'class':'serp-item__title'}).get_text(), # Название вакансии
                'link': item.find('a', {'class':'serp-item__title'}).get('href'), # Ссылка на вакансию
                'salary_min': salary_prep(salary)[0], # Зарплата минимальная
                'salary_max': salary_prep(salary)[1], # Зарплата максимальная
                'salary_currency': salary_prep(salary)[2], # Валюта зарплаты
                'city': text_prep(item.find('div', {'data-qa':'vacancy-serp__vacancy-address'}).get_text()), # Город
                'organization': text_prep(item.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}).get_text()), # Наименование компании
                }
            )

    return hh_vacancy

In [43]:
# Функция парсинга
def parser_hh():
    post = str(input('Введите название вакансии для парсинга: '))
    pages = int(input('Количество страниц для парсинга: '))
    html = resp
    if html.status_code == 200:
        vacancy = []
        for page in range(1, pages + 1):
            print(f'Парсим страницу {page}')
            html = requests.get(url_hh, headers=headers, params={'text': post, 'page': page})
            vacancy.extend(get_conten_hh(html))
        hh_result = pd.DataFrame(vacancy)
    else:
        hh_result = html.status_code
    
    hh_result.to_csv (r'vacancy.csv', index= False )
   
    return hh_result



In [45]:
parser_hh()

Введите название вакансии для парсинга: водитель
Количество страниц для парсинга: 3
Парсим страницу 1
Парсим страницу 2
Парсим страницу 3


,site,title,link,salary_min,salary_max,salary_currency,city,organization
0,HeadHanter,Водитель категории Е,https://adsrv.hh.ru/click?b=329606&c=30&meta=A...,NaN,100000.0,руб.,Омск,МАГНИТ
1,HeadHanter,Водитель на самовал,https://adsrv.hh.ru/click?b=367192&c=30&meta=I...,80000.0,100000.0,руб.,Тула,ООО ТСК
2,HeadHanter,Водитель-курьер с личным легковым автомобилем,https://lyubertsy.hh.ru/vacancy/77143606?from=...,90000.0,130000.0,руб.,Москва,ТопДеливери
3,HeadHanter,Водитель Яндекс.Такси на АВТОМОБИЛЕ КОМПАНИИ,https://lyubertsy.hh.ru/vacancy/52493958?from=...,1300.0,2500.0,бел.руб.,Минск,ООО ПРАЙМ МОТОРС
4,HeadHanter,Водитель легкового автомобиля (такси),https://lyubertsy.hh.ru/vacancy/76399462?from=...,90000.0,NaN,руб.,Санкт-Петербург,ООО Вк Групп
5,HeadHanter,Водитель-курьер на личном автомобиле (Кемерово),https://lyubertsy.hh.ru/vacancy/76962474?from=...,75000.0,NaN,руб.,Кемерово,Ozon Производство
6,HeadHanter,Водитель-курьер (Разные районы города),https://lyubertsy.hh.ru/vacancy/75169942?from=...,NaN,130000.0,руб.,Санкт-Петербург,ТОКИО-CITY
7,HeadHanter,Водитель штатный на автомобиль компании,https://lyubertsy.hh.ru/vacancy/76966722?from=...,NaN,200000.0,руб.,Москва,Trust Human Resources
8,HeadHanter,Водитель-курьер на личном автомобиле (Гоголя ул.),https://lyubertsy.hh.ru/vacancy/77015906?from=...,67500.0,NaN,руб.,Череповец,Ozon Производство
9,HeadHanter,Водитель-международник-стажёр,https://lyubertsy.hh.ru/vacancy/69183710?from=...,1200.0,NaN,EUR,Минск,R Group


In [17]:
soup = bs(resp.text, 'html.parser')
items = soup.find_all('div', {'class':'vacancy-serp-item__layout'})
hh_vacancy = []

for item in items:
    salary = item.find('span', {'class':'bloko-header-section-3'}).get_text()
    
    hh_vacancy.append(
        {
            'site': 'HeadHanter', # Название сайта
            'title': item.find('a', {'class':'serp-item__title'}).get_text(), # Название вакансии
            'link': item.find('a', {'class':'serp-item__title'}).get('href'), # Ссылка на вакансию
            'salary_min': salary_prep(salary)[0], # Зарплата минимальная
            'salary_max': salary_prep(salary)[1], # Зарплата максимальная
            'salary_currency': salary_prep(salary)[2], # Валюта зарплаты
            'city': text_prep(item.find('div', {'data-qa':'vacancy-serp__vacancy-address'}).get_text()), # Город
            'organization': text_prep(item.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}).get_text()), # Наименование компании
            }
        )
    
hh_vacancy
    print(type(salary))
# salary = salary.replace("\u202f", "")
# print(salary)
# salary.split()
# salary_prep(salary)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
